In [3]:
import os
import sys
from pathlib import Path

from IPython.display import display, HTML, Markdown
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Project level imports
sys.path.insert(0, '../lib')
from ncbi_remap.notebook import Nb
from ncbi_remap.plotting import make_figs

In [4]:
# Setup notebook
nbconfig = Nb.setup_notebook()

Please check output/fbgn2chrom.tsv. If it does not exist, run bin/fbgn2chrom.py
last updated: 2018-10-08 
Git hash: c5425d3c436714fc2475c8e5525cbd28738fd3eb


In [5]:
# Connect to data store
store = pd.HDFStore('../sra.h5', mode='r')

In [10]:
complete = store['aln/complete']
complete_srx = complete['srx'].unique().tolist()

In [7]:
from pymongo import MongoClient
try:
    with open('../output/.mongodb_host', 'r') as fh:
        host = fh.read().strip()
except FileNotFoundError:
    host = 'localhost'

mongoClient = MongoClient(host=host, port=27017)
db = mongoClient['sra']
ncbi = db['ncbi']

In [11]:
libstrat = pd.DataFrame(list(ncbi.aggregate([
    {
        '$match': {
            '_id': {'$in': complete_srx}
        }
    },
    {
        '$project': {
            'srx': '$_id',
            '_id': 0,
            'library_strategy': '$sra.experiment.library_strategy'
        }
    }
])))
libstrat.set_index('srx', inplace=True)

In [44]:
freetxt = pd.read_parquet('../metadata-wf/output/free_text_library_strategy.parquet')

In [49]:
dat = libstrat.join(freetxt).dropna()

In [50]:
(dat.library_strategy == dat.freetxt_library_strategy).sum() / dat.shape[0] * 100

60.407116774143418

In [51]:
diff = dat[(dat.library_strategy != dat.freetxt_library_strategy)]

In [52]:
diff.freetxt_library_strategy.value_counts()

3Prime-Seq|CLONE|DNA-Seq|Kilo-Seq     4158
END-Seq|RNA-Seq                        771
4C-Seq                                 721
ChIP-Seq|RNA-Seq                       588
END-Seq                                318
mmPCR-Seq                              304
3Prime-Seq|RNA-Seq                     287
RNA-Seq                                272
3Prime-Seq                             265
3Prime-Seq|WGS                         212
RNA-Seq|mmPCR-Seq                      117
WGS                                    108
ChIP-Seq|MNase-Seq                      83
END-Seq|WGS                             78
ATAC-Seq|END-Seq                        78
4C-Seq|HiC-Seq                          78
Pool-Seq                                77
END-Seq|Pool-Seq                        77
DNA-Seq|WGS                             73
DNA-Seq                                 62
TADA-Seq                                59
3Prime-Seq|CLONE|EST|RNA-Seq            58
RAD-Seq|WGS                             55
ATAC-Seq|Ch

In [55]:
diff[diff.library_strategy == 'RNA-Seq'].freetxt_library_strategy.value_counts()

END-Seq|RNA-Seq                                              456
ChIP-Seq|RNA-Seq                                             433
3Prime-Seq|RNA-Seq                                           259
END-Seq                                                      212
RNA-Seq|mmPCR-Seq                                            113
3Prime-Seq|CLONE|EST|RNA-Seq                                  58
WGS                                                           53
RNA-Seq|TAIL-Seq|mTAIL-Seq                                    33
3Prime-Seq|ChIP-Seq|END-Seq|RNA-Seq|RNaseH-Seq                32
GRO-Seq|RNA-Seq                                               30
CLIP-Seq|RNA-Seq                                              24
IP-Seq|RNA-Seq                                                24
FAIRE-Seq|RNA-Seq                                             21
ATAC-Seq|RNA-Seq|Start-Seq                                    20
DNA-Seq|END-Seq|Nascent-Seq|RNA-Seq                           18
4C-Seq|RNA-Seq           